# manage packages with pip

In [13]:
!which pip

/Users/lizre/.pyenv/versions/3.7.3/bin/pip


(if you're in a virtual environment, path will include “envs/env-name”)

Starts by searching [PyPI](https://pypi.org/): py package index

In [ ]:
!pip install pandas # Will also get dependencies needed for pandas
!pip list # Should now include pandas

dependencies and dependents:

In [86]:
pandas_deps = !pip show pandas 

In [89]:
pandas_deps

['Name: pandas',
 'Version: 1.3.2',
 'Summary: Powerful data structures for data analysis, time series, and statistics',
 'Home-page: https://pandas.pydata.org',
 'Author: The Pandas Development Team',
 'Author-email: pandas-dev@python.org',
 'License: BSD-3-Clause',
 'Location: /Users/lizre/.pyenv/versions/3.7.3/lib/python3.7/site-packages',
 'Requires: numpy, python-dateutil, pytz',
 'Required-by: cmdstanpy, mlflow, mlxtend, octopy, pandas-profiling, phik, prophet, researchpy, seaborn, sklearn-pandas, statsmodels, visions']

# create requirements.txt 

In terminal:

In [8]:
!pipreqs --force

INFO: Successfully saved requirements file in /Users/lizre/Downloads/learn-py/requirements.txt


- You'll see requirements.txt appear in src. It will not exactly match everything you `import` because pipreqs only includes ones not in standard library.
- Can also use `pip freeze > requirements.txt` but it is [harmful because it includes too many things.](https://medium.com/@tomagee/pip-freeze-requirements-txt-considered-harmful-f0bce66cf895); also [pypi says not to use pip freeze](https://pypi.org/project/pipreqs/).



In [10]:
!cat requirements.txt

pandas==1.3.2


To use:

In [12]:
!pip install -r requirements.txt

# Virtual environment

### Problem/motivation

When you pip install a package, it installs the dependency together with your interpreter. The package ends up in the Scripts folder in the directory where you installed Python. This is your environment: Python 3.7 with all of the installed packages.

So each Python project on your computer will install all their dependencies together with every other project.

See where we have python:

In [16]:
!which python

/Users/lizre/.pyenv/versions/3.7.3/bin/python


Inside that folder are all our packages:

In [82]:
my_global_python_packages = !cd ~/.pyenv/versions/3.7.3/lib/python3.7/site-packages && ls

In [73]:
import pandas as pd
my_global_python_packages = pd.Series(my_global_python_packages)

In [91]:
my_global_python_packages[200:210]

['itsdangerous',
 'itsdangerous-2.0.1.dist-info',
 'jedi',
 'jedi-0.18.0.dist-info',
 'jeepney',
 'jeepney-0.6.0.dist-info',
 'jinja2',
 'jinjasql',
 'jinjasql-0.1.8.dist-info',
 'jmespath']

In [83]:
my_global_python_packages[303]

'pandas'


Becomes problem when you have multiple projects that need different package versions. Each project will be looking at this same list of packages.


### Solution
A **virtual environment** with its own Python interpreter (`lib/python`), pip executable, and `site-packages` directory. Even its own pip.

Isolated from the “system” Python shown above. Also isolated from other virtual environments.

Also good to share code with someone who might have different packages and versions.

**Aside: why not docker:**

virtualenv only encapsulates Python dependencies. A Docker container encapsulates an entire OS. use docker when large-scale and/or dpeloying to a server.

## Make an env

[Beginners, start with venv.](https://stackoverflow.com/questions/41573587/what-is-the-difference-between-venv-pyvenv-pyenv-virtualenv-virtualenvwrappe). But many people prefer [pipenv](https://medium.com/analytics-vidhya/why-pipenv-over-venv-for-python-projects-a51fb6e4f31e).


In [32]:
!python3 -m venv my_env

In [15]:
!cd my_env && ls

bin        include    lib        pyvenv.cfg


Env will contain a copy of whatever python interpreter you used to make it. so make it specifically with `python3`.

creates a folder in the current directory which will contain the Python executable files, and a copy of the pip library which you can use to install other package:

In [ ]:
├── bin
│   ├── activate
│   ├── activate.csh
│   ├── activate.fish
│   ├── easy_install
│   ├── pip
│   ├── pip3
│   ├── python -> python3.7
│   └── python3.7 -> /Library/Frameworks/Python.framework/Versions/3.5/bin/python3.5
├── include
├── lib
│   └── python3.7
│       └── site-packages
└── pyvenv.cfg

- "activate" scripts set up your shell to use the environment’s Python executable and its site-packages by default.
- lib: a copy of the Python version. 

virtualenv --no-site-packages will not include the packages that are installed globally. This can be useful for keeping the package list clean in case it needs to be accessed later. [This is the default behavior for virtualenv 1.7 and later.]

Or conda: `!conda create --name example-name python=3.7 --no-default-packages`

## Enter/activate env

by running an activate script in the virtual environment’s executables directory 

In [19]:
!source my_env/bin/activate

Or conda: `!conda activate example-name`

Now prompt has env name:

![image](https://user-images.githubusercontent.com/38010821/153217272-690c3c2d-7035-474b-88ee-3ba1238a2d21.png)


`bin` holds the modules available in our venv:

In [33]:
!cd my_env/bin && ls

activate         easy_install     pip3             python3
activate.csh     easy_install-3.7 pip3.7
activate.fish    pip              python


`lib` &  `site-packages`: holds dependencies/packages you install in the venv:

In [34]:
!cd my_env/lib/python3.7/site-packages && ls

__pycache__                 pkg_resources
easy_install.py             setuptools
pip                         setuptools-40.8.0.dist-info
pip-19.0.3.dist-info


In [35]:
!source my_env/bin/activate && pip install numpy

  Using cached https://files.pythonhosted.org/packages/09/8c/ae037b8643aaa405b666c167f48550c1ce6b7c589fe5540de6d83e5931ca/numpy-1.21.5-cp37-cp37m-macosx_10_9_x86_64.whl
You are using pip version 19.0.3, however version 22.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


Now numpy's there:

In [36]:
!cd my_env/lib/python3.7/site-packages && ls

__pycache__                 pip-19.0.3.dist-info
easy_install.py             pkg_resources
numpy                       setuptools
numpy-1.21.5.dist-info      setuptools-40.8.0.dist-info
pip


#### How it works 

tricks the shell that the only python and pip executables are the ones in the virtual environment

When a virtual environment is active (i.e., the virtual environment’s Python interpreter is running), the attributes sys.prefix and sys.exec_prefix point to the base directory of the virtual environment, whereas sys.base_prefix and sys.base_exec_prefix point to the non-virtual environment Python installation which was used to create the virtual environment. If a virtual environment is not active, then sys.prefix is the same as sys.base_prefix and sys.exec_prefix is the same as sys.base_exec_prefix (they all point to a non-virtual environment Python installation).

scripts installed into virtual environments have a “shebang” line which points to the virtual environment’s Python interpreter. This means that the script will run with that interpreter regardless of the value of PATH.

## Exit env

Now your prompt will have (example-name) before it

In [ ]:
deactivate


To delete a virtual environment, just delete its folder.

more

[envs inside jupyter](https://stephen-odaibo.medium.com/docker-containers-python-virtual-environments-virtual-machines-d00aa9b8475)
[use cookiecutter](https://www.datacamp.com/community/tutorials/python-developer-set-up) for scaffolding/templating